## Set up and Load Data

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from sklearn.linear_model import ElasticNet, SGDRegressor, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [3]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr
from sklearn.metrics import make_scorer

In [4]:
# load labels
train_label = np.load('train_label.npy')
test_label = np.load('test_label.npy')

In [5]:
train_label[:5]

array([34.20739619, 26.45371972, 34.96756055, 22.04476644, 25.84558824])

In [6]:
test_label[:5]

array([29.69849537, 30.84591837, 24.38979592, 36.25867883, 27.89129141])

In [7]:
# Load vgg16 features before relu: should have negative value
vgg16_train_features = np.load('vgg16_train_features.npy')
vgg16_test_features = np.load('vgg16_test_features.npy')

In [8]:
vgg16_train_features[:5]

array([[ -1.4681634 ,  -0.5275203 , -22.067028  , ..., -17.184917  ,
        -11.0921135 ,  -0.35512644],
       [ -0.20396854,  -0.71452796, -62.699623  , ..., -21.611687  ,
        -34.978123  ,  -0.25105998],
       [ -0.8742988 ,  -0.51126605, -30.323936  , ...,  -3.6527483 ,
         -9.731274  ,  -0.28503752],
       [ -0.71330154,  -0.4714042 , -33.486244  , ..., -10.589474  ,
        -10.808492  ,  -0.24790464],
       [  0.9988323 ,  -0.32367784, -21.522228  , ...,   4.6680403 ,
          6.1433697 ,  -0.23947598]], dtype=float32)

In [9]:
vgg16_test_features[:5]

array([[ -3.9410076 ,  -0.6707838 , -53.463104  , ...,  -8.278335  ,
        -10.292274  ,  -0.265556  ],
       [ -0.1634852 ,  -0.5534688 , -22.074677  , ...,   5.6482077 ,
         18.613846  ,  -0.42996874],
       [  1.8922055 ,  -0.8144202 , -27.296654  , ...,   8.652847  ,
          0.21162796,  -0.26159078],
       [ -1.96607   ,  -0.30473575,  -4.8362856 , ...,   2.0165455 ,
         -1.1177328 ,  -0.26760662],
       [ -1.2365555 ,  -0.18508121,   9.221886  , ...,   0.6952783 ,
         -2.750974  ,  -0.17681438]], dtype=float32)

In [10]:
# Load vgg16 features after relu: should not contain negative value
vgg16_relu_train_features = np.load('vgg16_relu_train_features.npy')
vgg16_relu_test_features = np.load('vgg16_relu_test_features.npy')

In [11]:
vgg16_relu_train_features[:5]

array([[0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.9988323, 0.       , 0.       , ..., 4.6680403, 6.1433697,
        0.       ]], dtype=float32)

In [12]:
vgg16_relu_test_features[:5]

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  5.6482077 ,
        18.613846  ,  0.        ],
       [ 1.8922055 ,  0.        ,  0.        , ...,  8.652847  ,
         0.21162796,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  2.0165455 ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  9.221886  , ...,  0.6952783 ,
         0.        ,  0.        ]], dtype=float32)

In [13]:
# Load resnet features
resnet_train_features = np.load('resnet_train_features.npy')
resnet_test_features = np.load('resnet_test_features.npy')

In [14]:
resnet_train_features[:5] 

array([[[[0.07640424, 0.11314762, 0.7201315 , ..., 7.5590777 ,
          2.6926625 , 0.        ]]],


       [[[0.33827135, 1.4292572 , 0.        , ..., 5.61894   ,
          0.        , 0.        ]]],


       [[[0.01324131, 6.6258082 , 0.        , ..., 0.        ,
          0.        , 7.274989  ]]],


       [[[0.12476952, 2.2230198 , 4.2002983 , ..., 0.56822747,
          0.        , 0.20838165]]],


       [[[0.7693701 , 0.        , 0.26286367, ..., 1.2396903 ,
          1.1977144 , 0.        ]]]], dtype=float32)

In [15]:
# Flatten the features for SVR
resnet_train_features_flatten = resnet_train_features.reshape(resnet_train_features.shape[0], -1)
resnet_test_features_flatten = resnet_test_features.reshape(resnet_test_features.shape[0], -1)

In [16]:
# Load senet features
senet_train_features = np.load('senet_train_features.npy')
senet_test_features = np.load('senet_test_features.npy')

senet_train_features[:5]

In [17]:
# Flatten the features for SVR
senet_train_features_flatten = senet_train_features.reshape(senet_train_features.shape[0], -1)
senet_test_features_flatten = senet_test_features.reshape(senet_test_features.shape[0], -1)

## Regression


#### UDF

In [20]:
def evaluate_performance(model, X_test, y_test):
    predicted_values = model.predict(X_test)
    
    rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
    mae = mean_absolute_error(y_test, predicted_values)
    corr = np.corrcoef(y_test, predicted_values)[0][1]
    
    print("RMSE: ", rmse)
    print("MAE: ", mae)
    print("Pearson Correlation: ", corr)

In [21]:
def get_performance(model, X_test, y_test):
    
    predicted_values = model.predict(X_test)
    
    rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
    mae = mean_absolute_error(y_test, predicted_values)
    corr = np.corrcoef(y_test, predicted_values)[0][1]
    
    return {'RMSE': rmse, 'MAE': mae, 'Pearson Correlation': corr}


In [22]:
# Define a custom scorer
def pearson_corr(y_true, y_pred):
    corr = np.corrcoef(y_true, y_pred)[0][1]
    return corr

In [23]:
# Create a scorer from the custom scorer function
pearson_scorer = make_scorer(pearson_corr)

#### Test to Estimate the Time

##### Linear

In [24]:
%%time
test_linear = LinearRegression()
test_linear.fit(vgg16_train_features, train_label)

CPU times: user 38.4 s, sys: 20.6 s, total: 59 s
Wall time: 4.58 s


LinearRegression()

In [34]:
evaluate_performance(test_linear, vgg16_test_features, test_label)

RMSE:  21.13836624699798
MAE:  16.68405416725854
Pearson Correlation:  0.26545901293000457


In [65]:
test_linear.fit(vgg16_relu_train_features, train_label)
evaluate_performance(test_linear, vgg16_relu_test_features, test_label)

RMSE:  19.946731053772822
MAE:  15.540127252773123
Pearson Correlation:  0.1930424857734299


In [70]:
test_linear.fit(resnet_train_features_flatten, train_label)
evaluate_performance(test_linear, resnet_test_features_flatten, test_label)

RMSE:  10.541088583654513
MAE:  8.323345441586486
Pearson Correlation:  0.35175197248378237


In [71]:
test_linear.fit(senet_train_features_flatten, train_label)
evaluate_performance(test_linear, senet_test_features_flatten, test_label)

RMSE:  9.838797537183734
MAE:  7.728548507586659
Pearson Correlation:  0.4671883035006795


##### Elastic Net

In [37]:
%%time
test_en = ElasticNet()
test_en.fit(vgg16_train_features, train_label)

CPU times: user 7.44 s, sys: 924 ms, total: 8.36 s
Wall time: 7.97 s


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.216e+02, tolerance: 2.063e+01
  model = cd_fast.enet_coordinate_descent(


ElasticNet()

In [38]:
evaluate_performance(test_en, vgg16_test_features, test_label)

RMSE:  7.014683069544612
MAE:  5.050161915369192
Pearson Correlation:  0.6539637462914791


In [66]:
test_en.fit(vgg16_relu_train_features, train_label)
evaluate_performance(test_en, vgg16_relu_test_features, test_label)

RMSE:  7.115130229344088
MAE:  5.087043406347265
Pearson Correlation:  0.6414804542547985


In [72]:
test_en.fit(resnet_train_features_flatten, train_label)
evaluate_performance(test_en, resnet_test_features_flatten, test_label)

RMSE:  7.1822421966154355
MAE:  5.144145567766589
Pearson Correlation:  0.6401847637125766


In [73]:
test_en.fit(senet_train_features_flatten, train_label)
evaluate_performance(test_en, senet_test_features_flatten, test_label)

RMSE:  6.933255800783864
MAE:  4.877885629556298
Pearson Correlation:  0.6683680711477811


##### Random Forest

In [28]:
"""
%%time
test_rfr = RandomForestRegressor()
test_rfr.fit(vgg16_train_features, train_label)
"""
# too long

'\n%%time\ntest_rfr = RandomForestRegressor()\ntest_rfr.fit(vgg16_train_features, train_label)\n'

##### XGBoost

In [31]:
%%time
test_xgb = XGBRegressor()
test_xgb.fit(vgg16_train_features, train_label)

CPU times: user 24min 2s, sys: 1.23 s, total: 24min 3s
Wall time: 3min 19s


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [33]:
evaluate_performance(test_xgb, vgg16_test_features, test_label)

RMSE:  7.381432068425613
MAE:  5.294061220355851
Pearson Correlation:  0.6032810470133607


In [67]:
test_xgb.fit(vgg16_relu_train_features, train_label)
evaluate_performance(test_xgb, vgg16_relu_test_features, test_label)

RMSE:  7.516790975991106
MAE:  5.430389682177232
Pearson Correlation:  0.5861548885869396


In [74]:
test_xgb.fit(resnet_train_features_flatten, train_label)
evaluate_performance(test_xgb, resnet_test_features_flatten, test_label)

RMSE:  7.59879070885557
MAE:  5.475492957439652
Pearson Correlation:  0.5732723975261663


In [75]:
test_xgb.fit(senet_train_features_flatten, train_label)
evaluate_performance(test_xgb, senet_test_features_flatten, test_label)

RMSE:  7.23517286489358
MAE:  5.267563988621195
Pearson Correlation:  0.6245218541967646


##### LightGBM

In [35]:
%%time
test_lgb = LGBMRegressor()
test_lgb.fit(vgg16_train_features, train_label)

CPU times: user 2min 7s, sys: 4.88 s, total: 2min 12s
Wall time: 25.3 s


LGBMRegressor()

In [36]:
evaluate_performance(test_lgb, vgg16_test_features, test_label)

RMSE:  7.143315595715149
MAE:  5.047451333340878
Pearson Correlation:  0.6393910936241822


In [68]:
test_lgb.fit(vgg16_relu_train_features, train_label)
evaluate_performance(test_lgb, vgg16_relu_test_features, test_label)

RMSE:  7.250339905223896
MAE:  5.1605693781404405
Pearson Correlation:  0.6227300868696897


In [76]:
test_lgb.fit(resnet_train_features_flatten, train_label)
evaluate_performance(test_lgb, resnet_test_features_flatten, test_label)

RMSE:  7.177524972146452
MAE:  5.145443639438138
Pearson Correlation:  0.6335467001047109


In [77]:
test_lgb.fit(senet_train_features_flatten, train_label)
evaluate_performance(test_lgb, senet_test_features_flatten, test_label)

RMSE:  6.980006244247765
MAE:  4.927319912675946
Pearson Correlation:  0.6587930116086919


##### Stochastic Gradient Descent

In [80]:
%%time
test_sgd = SGDRegressor()
test_sgd.fit(vgg16_train_features, train_label)

CPU times: user 8.17 s, sys: 0 ns, total: 8.17 s
Wall time: 8.17 s


SGDRegressor()

In [81]:
evaluate_performance(test_sgd, vgg16_test_features, test_label)

RMSE:  454273112071762.9
MAE:  361704156818112.75
Pearson Correlation:  0.021703539728922604


In [69]:
test_sgd.fit(vgg16_relu_train_features, train_label)
evaluate_performance(test_sgd, vgg16_relu_test_features, test_label)

RMSE:  78833104339915.23
MAE:  62987831506264.74
Pearson Correlation:  -0.2037262524034574


In [78]:
test_sgd.fit(resnet_train_features_flatten, train_label)
evaluate_performance(test_sgd, resnet_test_features_flatten, test_label)

RMSE:  2118747260268.2974
MAE:  1715661444687.0852
Pearson Correlation:  -0.03707558980368991


In [79]:
test_sgd.fit(senet_train_features_flatten, train_label)
evaluate_performance(test_sgd, senet_test_features_flatten, test_label)

RMSE:  6058294676746.038
MAE:  4885715860741.062
Pearson Correlation:  -0.026901076331921087


#### Grid Search

In [18]:
# list of feature sets
feature_sets = [(vgg16_train_features, train_label, vgg16_test_features, test_label), 
                (vgg16_relu_train_features, train_label, vgg16_relu_test_features, test_label),
                (resnet_train_features_flatten, train_label, resnet_test_features_flatten, test_label),
                (senet_train_features_flatten, train_label, senet_test_features_flatten, test_label)]

# models
models = [LinearRegression(), ElasticNet(), SGDRegressor(), LGBMRegressor()]

# Parameter grids for GridSearch
param_grids = [
                # for LinearRegression
                {}, 
                # for ElasticNet
                # {"alpha": [0.1, 0.5], "l1_ratio": [0.1, 0.5, 0.7, 0.9, 1], "max_iter": [10000, 100000]},
                {},
                # for SGDRegressor
                # {"penalty": ["l2", "l1", "elasticnet"], "alpha": [0.0001, 0.001, 0.01, 0.1], "l1_ratio": [0, 0.3, 0.5, 0.7, 0.9], "max_iter": [10000, 100000]},
                {},
                # for LGBMRegressor
                {"n_estimators": [100, 200], "learning_rate": [0.01, 0.05], "max_depth": [3, 5], "subsample": [0.5, 0.7]}
               ]


In [24]:
%%time
# Creating a DataFrame to store results
results_df = pd.DataFrame(columns=['Linear Regression', 'ElasticNet', 'SGDRegressor', 'LightGBM'])
indices = ['vgg16_fc6', 'vgg16_fc6_relu', 'resnet', 'senet']

for i, feature_set in enumerate(feature_sets):
    X_train, y_train, X_test, y_test = feature_set
    for j, model in enumerate(models):
        # Performing grid search
        grid_search = GridSearchCV(model, param_grids[j], cv=5, scoring=pearson_scorer)
        grid_search.fit(X_train, y_train)
        
        # Evaluating the best model
        best_model = grid_search.best_estimator_
        performance = get_performance(best_model, X_test, y_test)
        
        # Storing the performance
        results_df.loc[indices[i], type(best_model).__name__] = str(performance)

print(results_df)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.591e+02, tolerance: 1.687e+01
  model = cd_fast.enet_coordinate_descent(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.514e+02, tolerance: 1.635e+01
  model = cd_fast.enet_coordinate_descent(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.643e+02, tolerance: 1.682e

               Linear Regression  \
vgg16_fc6                    NaN   
vgg16_fc6_relu               NaN   
resnet                       NaN   
senet                        NaN   

                                                       ElasticNet  \
vgg16_fc6       {'RMSE': 7.014683069544612, 'MAE': 5.050161915...   
vgg16_fc6_relu  {'RMSE': 7.115130229344088, 'MAE': 5.087043406...   
resnet          {'RMSE': 7.1822421966154355, 'MAE': 5.14414556...   
senet           {'RMSE': 6.933255800783864, 'MAE': 4.877885629...   

                                                     SGDRegressor LightGBM  \
vgg16_fc6       {'RMSE': 494840861209124.6, 'MAE': 39456776927...      NaN   
vgg16_fc6_relu  {'RMSE': 49032426527351.26, 'MAE': 38948741778...      NaN   
resnet          {'RMSE': 2174310941556.2698, 'MAE': 1759800151...      NaN   
senet           {'RMSE': 6530466976782.726, 'MAE': 52050813880...      NaN   

                                                 LinearRegression  \
vgg16_fc6    

In [25]:
results_df

Linear Regression  \
vgg16_fc6                    NaN   
vgg16_fc6_relu               NaN   
resnet                       NaN   
senet                        NaN   

                                                       ElasticNet  \
vgg16_fc6       {'RMSE': 7.014683069544612, 'MAE': 5.050161915...   
vgg16_fc6_relu  {'RMSE': 7.115130229344088, 'MAE': 5.087043406...   
resnet          {'RMSE': 7.1822421966154355, 'MAE': 5.14414556...   
senet           {'RMSE': 6.933255800783864, 'MAE': 4.877885629...   

                                                     SGDRegressor LightGBM  \
vgg16_fc6       {'RMSE': 494840861209124.6, 'MAE': 39456776927...      NaN   
vgg16_fc6_relu  {'RMSE': 49032426527351.26, 'MAE': 38948741778...      NaN   
resnet          {'RMSE': 2174310941556.2698, 'MAE': 1759800151...      NaN   
senet           {'RMSE': 6530466976782.726, 'MAE': 52050813880...      NaN   

                                                 LinearRegression  \
vgg16_fc6       {'RMSE': 21.13836624699798, 'MAE': 16.68405416...   
vgg16_fc6_relu  {'RMSE': 19.946731053772822, 'MAE': 15.5401272...   
resnet          {'RMSE': 10.541088583654513, 'MAE': 8.32334544...   
senet           {'RMSE': 9.838797537183734, 'MAE': 7.728548507...   

                                                    LGBMRegressor  
vgg16_fc6       {'RMSE': 7.073911385466153, 'MAE': 5.016375929...  
vgg16_fc6_relu  {'RMSE': 7.143275324210219, 'MAE': 5.056316688...  
resnet          {'RMSE': 7.157644624444229, 'MAE': 5.118883468...  
senet           {'RMSE': 6.938628110878432, 'MAE': 4.918025269...

In [27]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [28]:
results_df['LGBMRegressor']

vgg16_fc6          {'RMSE': 7.073911385466153, 'MAE': 5.01637592968152, 'Pearson Correlation': 0.6509743612257722}
vgg16_fc6_relu     {'RMSE': 7.143275324210219, 'MAE': 5.05631668828041, 'Pearson Correlation': 0.6410626551704808}
resnet            {'RMSE': 7.157644624444229, 'MAE': 5.118883468920799, 'Pearson Correlation': 0.6364474995548529}
senet             {'RMSE': 6.938628110878432, 'MAE': 4.918025269400238, 'Pearson Correlation': 0.6672503222226467}
Name: LGBMRegressor, dtype: object